## PART 1 OF THE FINAL ASSIGNMENT

In [82]:
import sys
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install lxml

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lxml import html
from urllib.request import urlopen
import json
import numpy as np

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
df_canada = pd.read_html(url, header=0)[0]

df_canada.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [5]:
df_canada = df_canada[df_canada['Borough'] != 'Not assigned']
df_canada

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [6]:
df_canada.shape # removing not assigned items from

(103, 3)

# PART 2 OF THE FINAL ASSIGNMENT

In [7]:
!{sys.executable} -m pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 5.2MB/s ta 0:00:011


lat_lng_cords = None
lat_list = []
long_list = []

for index, row in df_canada.iterrows():
    postal_code = row["Postal Code"]
    while (lat_lng_cords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        print(g)
        lat_lng_coords = g.latlng
        print(lat_lng_cords)
        
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    lat_list.append(latitude)
    long_list.append(longitude)
    lat_lng_cords = None
    
print(lat_list)
print(long_list)

In [8]:
df_coords = pd.read_csv('Geospatial_Coordinates.csv')

df_coords.shape

(103, 3)

In [9]:
df_canada = df_canada.merge(df_coords, on = "Postal Code", how = "inner")
df_canada

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# PART 3 OF THE FINAL ASSIGNMENT

In [10]:
import requests

In [26]:
client_id = 'QMV2FFETPVLE2TKKMSJ4JSSUJXQ1DHRH4YGRHJPOGHVRTF4H'
client_secret = 'FQYCHIUUFNX1R4SGJRV43WBRVFLWVH0IXS4WSWVRCTOZKAZY'
version = '20180605'
lat = ""
lon = ""
radius = 500
limit = 100

In [27]:
def url2_creator(client_id, client_secret, version, lat, lon, radius, limit):
    url2 = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(client_id, client_secret, version, lat, lon, radius, limit)
           #'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude, neighborhood_longitude,radius, LIMIT)
    return url2

In [28]:
lat = df_canada.loc[0, "Latitude"]
lon = df_canada.loc[0, "Longitude"]
print(lat, lon)

43.7532586 -79.3296565


In [29]:
url2 = url2_creator(client_id, client_secret, version, lat, lon, radius, limit)
url2

'https://api.foursquare.com/v2/venues/explore?&client_id=QMV2FFETPVLE2TKKMSJ4JSSUJXQ1DHRH4YGRHJPOGHVRTF4H&client_secret=FQYCHIUUFNX1R4SGJRV43WBRVFLWVH0IXS4WSWVRCTOZKAZY&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [92]:
response = requests.get(url2).json()
response["response"]["groups"][0]["items"]
pass

In [53]:
for venue in response["response"]["groups"][0]["items"]:
    print(venue["venue"]["name"])
    print(venue["venue"]["categories"][0]["name"])

South St. Burger
Burger Joint
Wingporium
Wings Joint
Dollarama
Discount Store
Artisano Bakery Café
Bakery
Healthy Planet
Supplement Shop
7-Eleven
Convenience Store
Subway
Sandwich Place
McDonald's
Fast Food Restaurant
Jim & Maria's No Frills
Grocery Store
RONA
Hardware Store
Royal Canadian Legion #210
Social Club
Koala Tan Tanning Salon & Sunless Spa
Tanning Salon
Once Upon A Child
Kids Store
Value Village
Thrift / Vintage Store
Kingsway Boxing Club
Gym
Burrito Boyz
Burrito Place


In [54]:
df_canada.shape

(103, 5)

In [55]:
borough = list(df_canada["Borough"])
latitude = list(df_canada["Latitude"])
longitude = list(df_canada["Longitude"])

borough_list = []
venue_name = []
venue_category = []

for p_code, lat, long in zip(borough, latitude, longitude):
    url2 = url2_creator(client_id, client_secret, version, lat, long, radius, limit)
    response = requests.get(url2).json()["response"]["groups"][0]["items"]
    
    for venue in response:
        borough_list.append(p_code)
        venue_name.append(venue["venue"]["name"])
        venue_category.append(venue["venue"]["categories"][0]["name"])

In [56]:
dict_venues = {"Borough": borough_list, "Name": venue_name, "Category": venue_category}
#dict_venues

In [58]:
df_venues = pd.DataFrame(dict_venues)
df_venues.head()

,Borough,Name,Category
0,North York,Brookbanks Park,Park
1,North York,Variety Store,Food & Drink Shop
2,North York,Victoria Village Arena,Hockey Arena
3,North York,Tim Hortons,Coffee Shop
4,North York,Portugril,Portuguese Restaurant


In [59]:
df_venues.drop("Name", axis = 1, inplace = True)
df_venues.head()

,Borough,Category
0,North York,Park
1,North York,Food & Drink Shop
2,North York,Hockey Arena
3,North York,Coffee Shop
4,North York,Portuguese Restaurant


In [69]:
canada_onehot = pd.get_dummies(df_venues["Category"])
canada_onehot

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2137,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2138,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2139,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
canada_onehot["Borough"] = df_venues["Borough"]
canada_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Borough
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,North York
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,North York
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,North York
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,North York
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,North York


In [71]:
fixed_columns = [canada_onehot.columns[-1]] + list(canada_onehot.columns[:-1])

canada_onehot = canada_onehot[fixed_columns]
canada_onehot

,Borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,North York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2136,Etobicoke,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2137,Etobicoke,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2138,Etobicoke,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2139,Etobicoke,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
canada_grouped = canada_onehot.groupby("Borough").mean().reset_index()
canada_grouped

#df_venues[[canada_grouped ["Borough"]]]

,Borough,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009615,...,0.000000,0.00,0.000000,0.000000,0.009615,0.000000,0.000000,0.000000,0.000000,0.009615
1,Downtown Toronto,0.000000,0.000801,0.000801,0.000801,0.000801,0.001603,0.001603,0.000801,0.013622,...,0.002404,0.00,0.011218,0.001603,0.004006,0.000000,0.006410,0.000000,0.000000,0.005609
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025210,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.008403,0.000000,0.000000,0.016807
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.012658,0.000000,0.000000,0.000000,0.012658
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.013514,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004149,0.000000,0.004149,0.000000,0.000000,0.000000,0.000000,0.000000,0.008299,...,0.000000,0.00,0.000000,0.004149,0.008299,0.000000,0.000000,0.000000,0.016598,0.000000
7,Scarborough,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011111,...,0.011111,0.00,0.000000,0.000000,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000
8,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.013072,0.000000,0.013072,0.000000,0.006536,0.000000,0.000000,0.013072
9,York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000


In [65]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [78]:
num_top_venues = 10

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    columns.append('{} Most Common Venue'.format(ind+1))

df_canada_sorted = pd.DataFrame(columns=columns)
df_canada_sorted["Borough"] = canada_grouped["Borough"]

for ind in np.arange(canada_grouped.shape[0]):
    df_canada_sorted.iloc[ind, 1:] = return_most_common_venues(canada_grouped.iloc[ind, :], num_top_venues)

df_canada_sorted

,Borough,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Sushi Restaurant,Restaurant,Dessert Shop,Bagel Shop,Italian Restaurant
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Seafood Restaurant
2,East Toronto,Coffee Shop,Greek Restaurant,Brewery,Italian Restaurant,Restaurant,Park,Ice Cream Shop,Pizza Place,American Restaurant,Bakery
3,East York,Coffee Shop,Bank,Intersection,Burger Joint,Sandwich Place,Sporting Goods Shop,Pizza Place,Park,Pharmacy,Indian Restaurant
4,Etobicoke,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Grocery Store,Gym,Fast Food Restaurant,Liquor Store,Café,Bakery
5,Mississauga,Coffee Shop,Hotel,Middle Eastern Restaurant,Intersection,Gym,Fried Chicken Joint,Mediterranean Restaurant,Burrito Place,Gas Station,American Restaurant
6,North York,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café
7,Scarborough,Bakery,Coffee Shop,Intersection,Bank,Fast Food Restaurant,Indian Restaurant,Chinese Restaurant,Breakfast Spot,Pizza Place,Mexican Restaurant
8,West Toronto,Café,Bar,Coffee Shop,Italian Restaurant,Restaurant,Bakery,Breakfast Spot,Bookstore,Pizza Place,Gift Shop
9,York,Park,Brewery,Trail,Tennis Court,Bus Line,Breakfast Spot,Sandwich Place,Discount Store,Hockey Arena,Field


In [88]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

kclusters = 5

canada_grouped_clustering = canada_grouped.drop("Borough", 1)
kmeans = KMeans(n_clusters = kclusters).fit(canada_grouped_clustering)

canada_labels = kmeans.labels_[0:10]

In [91]:
df_canada_sorted["kmeans_labels"] = canada_labels
df_canada_sorted

,Borough,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue,kmeans_labels
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Sushi Restaurant,Restaurant,Dessert Shop,Bagel Shop,Italian Restaurant,0
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Seafood Restaurant,0
2,East Toronto,Coffee Shop,Greek Restaurant,Brewery,Italian Restaurant,Restaurant,Park,Ice Cream Shop,Pizza Place,American Restaurant,Bakery,0
3,East York,Coffee Shop,Bank,Intersection,Burger Joint,Sandwich Place,Sporting Goods Shop,Pizza Place,Park,Pharmacy,Indian Restaurant,3
4,Etobicoke,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Grocery Store,Gym,Fast Food Restaurant,Liquor Store,Café,Bakery,4
5,Mississauga,Coffee Shop,Hotel,Middle Eastern Restaurant,Intersection,Gym,Fried Chicken Joint,Mediterranean Restaurant,Burrito Place,Gas Station,American Restaurant,2
6,North York,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café,0
7,Scarborough,Bakery,Coffee Shop,Intersection,Bank,Fast Food Restaurant,Indian Restaurant,Chinese Restaurant,Breakfast Spot,Pizza Place,Mexican Restaurant,3
8,West Toronto,Café,Bar,Coffee Shop,Italian Restaurant,Restaurant,Bakery,Breakfast Spot,Bookstore,Pizza Place,Gift Shop,0
9,York,Park,Brewery,Trail,Tennis Court,Bus Line,Breakfast Spot,Sandwich Place,Discount Store,Hockey Arena,Field,1


In [93]:
df_canada_sorted[df_canada_sorted["kmeans_labels"] == 0]

,Borough,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue,6 Most Common Venue,7 Most Common Venue,8 Most Common Venue,9 Most Common Venue,10 Most Common Venue,kmeans_labels
0,Central Toronto,Coffee Shop,Sandwich Place,Café,Park,Pizza Place,Sushi Restaurant,Restaurant,Dessert Shop,Bagel Shop,Italian Restaurant,0
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Seafood Restaurant,0
2,East Toronto,Coffee Shop,Greek Restaurant,Brewery,Italian Restaurant,Restaurant,Park,Ice Cream Shop,Pizza Place,American Restaurant,Bakery,0
6,North York,Coffee Shop,Clothing Store,Japanese Restaurant,Pizza Place,Sandwich Place,Restaurant,Park,Bank,Grocery Store,Café,0
8,West Toronto,Café,Bar,Coffee Shop,Italian Restaurant,Restaurant,Bakery,Breakfast Spot,Bookstore,Pizza Place,Gift Shop,0
